#### Codigo para Header e Footer

In [78]:
from reportlab.pdfgen import canvas
from reportlab.platypus import Paragraph, Frame, Table, TableStyle
from reportlab.lib import colors, units
from reportlab.lib.pagesizes import A4

pdf = canvas.Canvas("tuto2.pdf", pagesize=A4)
pdf.translate(units.cm, units.cm)

estilo_table = TableStyle([('FONT',(0,0),(-1,-1),'Times-Bold',10),
                           ()])
 
frameh = Frame(10, 770, 500, 30, showBoundary=1)
flow_obj1 = []
tableh = Table([['Luan Lucas']])
flow_obj1.append(tableh)
frameh.addFromList(flow_obj1, pdf)

largura, altura = A4
pdf.setFontSize(20)
pdf.drawCentredString(largura/2, 450, 'Texto contido em um drawCentredString')
pdf.drawCentredString(largura/2, altura/2, 'ABC')
pdf.drawCentredString(largura/2, 400, 'Texto contido em um drawCentredString')
pdf.drawCentredString(largura/2, 550, 'Texto contido em um drawCentredString')
pdf.drawCentredString(largura/2, 500, 'Texto contido em um drawCentredString')
# pdf.drawAlignedString(largura/2, altura/2, 'Texto contido em um drawAlignedString')

framef = Frame(10, 10, 500, 30, showBoundary=1)
flow_obj2 = []
tablef = Table([['Página '+str(pdf.getPageNumber())]])
tablef.setStyle(estilo_table)
flow_obj2.append(tablef)
framef.addFromList(flow_obj2, pdf)

pdf.save()

#### Codigo para Header e Footer com o PageTemplate

In [17]:
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib.units import cm
from reportlab.platypus import SimpleDocTemplate, PageTemplate
from reportlab.platypus.frames import Frame
from reportlab.lib import pagesizes
from reportlab.platypus.paragraph import Paragraph
from functools import partial

def header(canvas, doc, content):
    canvas.saveState()
    w, h = content.wrap(doc.width, doc.topMargin)
    content.drawOn(canvas, doc.leftMargin, doc.height + doc.bottomMargin + doc.topMargin - h)
    canvas.restoreState()

def footer(canvas, doc, content):
    canvas.saveState()
    w, h = content.wrap(doc.width, doc.bottomMargin)
    content.drawOn(canvas, doc.leftMargin, h)
    canvas.restoreState()

def header_and_footer(canvas, doc, header_content, footer_content):
    header(canvas, doc, header_content)
    footer(canvas, doc, footer_content)

styles = getSampleStyleSheet()

filename = "out.pdf"

PAGESIZE = pagesizes.portrait(pagesizes.A4)

pdf = SimpleDocTemplate(filename, pagesize=PAGESIZE, 
        leftMargin = 2.2 * cm, 
        rightMargin = 2.2 * cm,
        topMargin = 1.5 * cm, 
        bottomMargin = 2.5 * cm)

frame = Frame(pdf.leftMargin, pdf.bottomMargin, pdf.width, pdf.height, id='normal')

header_content = Paragraph("This is a header. testing testing testing  ", styles['Normal'])
footer_content = Paragraph("This is a footer. It goes on every page.  ", styles['Normal'])

template = PageTemplate(id='test', frames=frame, onPage=partial(header_and_footer, header_content=header_content, footer_content=footer_content))

pdf.addPageTemplates([template])

pdf.build([Paragraph("This is content")])

## Criando tabelas grandes

### Gerando uma planilha com mais de 1000 linhas

In [694]:
import random
from openpyxl import Workbook
from faker import Faker

# Crie um novo arquivo Excel (workbook)
wb = Workbook()

# Selecione a planilha ativa (a primeira planilha por padrão)
sheet = wb.active

# Adicione cabeçalhos às colunas
sheet.append(['Nome', 'Nascimento', 'Cargo', 'Salario', 'Empresa', 'N° de Telefone', 'Estado'])

# Crie uma instância do Faker para gerar dados falsos
fake = Faker()

# Gere números aleatórios e preencha as colunas
for _ in range(100):
    row_data = [
        fake.name(),                      # Nome
        fake.date_of_birth(minimum_age=18, maximum_age=65).strftime("%Y-%m-%d"),  # Nascimento
        fake.job(),                       # Cargo
        round(random.uniform(2000, 8000), 2),  # Salario (entre 2000 e 8000)
        fake.company(),                   # Empresa
        fake.phone_number(),              # Numero de telefone
        fake.state()                      # Estado
    ]
    sheet.append(row_data)

# Salve a planilha em um arquivo
wb.save("planilha_com_valores_aleatorios.xlsx")

# Feche o arquivo Excel
wb.close()


### Criando o PDF

In [766]:
from reportlab.pdfgen import canvas
from reportlab.lib import colors
from reportlab.platypus import Paragraph, Table, TableStyle, Frame, Spacer, Image
from reportlab.lib.units import cm
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.pagesizes import A4, landscape
import datetime

import pandas as pd

def centralizar_texto(pdf: canvas.Canvas, y_titulo, titulo, fontName, fontSize, espacamento=20):         
    # Calcular a posição X para centralizar o título
    largura, altura = landscape(A4)
    x_titulo = (largura) / 2

    # Configurar o título do relatório
    pdf.setFont(fontName, fontSize)

    # Desenhar o título centralizado
    pdf.drawCentredString(x_titulo, y_titulo, titulo)

    altura_pagina = y_titulo - espacamento  # Iniciar logo abaixo do título
    return altura_pagina

def converter_proporcao(largura_atual, altura_atual, nova_largura=None, nova_altura=None):
    '''
        Converter as dimensões de uma imagem
        
        return: (nova_largura, nova_altura)
    '''
    if nova_largura is not None:
        proporcao = nova_largura / largura_atual
        nova_altura = int(altura_atual * proporcao)
    elif nova_altura is not None:
        proporcao = nova_altura / altura_atual
        nova_largura = int(largura_atual * proporcao)

    return (nova_largura, nova_altura)

def definir_linhas(pdf,row,column):
    largura, altura = landscape(A4)
    # Height (Largura) da linha
    pdf.drawString(x=column/2, y=row, text=str(int(row)))
    pdf.line(x1=0, y1=row, x2=largura, y2=row)
    # Width (Altura) da linha
    pdf.drawString(x=column, y=row/2, text=str(int(column)))
    pdf.line(x1=column, y1=0, x2=column, y2=altura)

def inserir_imagem(nome_imagem: str, x: int, y: int, largura=None, altura=None, centered='XY', showBoundary=0):
    '''
        Inserir uma imagem
        
        return: (nova_largura, nova_altura)
    '''
    path=r'C:\Users\luan.pinto\Desktop\Códigos\Projeto - Controle de obras\IMAGE'+'\\'+nome_imagem
    image = Image(path,0,0)
    altura_imagem, largura_imagem = image.imageHeight, image.imageWidth
    # print(f'image "{nome_imagem}": ({image.imageWidth}, {image.imageHeight}): ')

    proporcao = converter_proporcao(image.imageWidth, image.imageHeight, largura, altura)
    largura_imagem, altura_imagem = proporcao

    if centered == 'XY': y = y-altura_imagem
    if centered == 'CENTER':
        x = (x-largura_imagem)/2
        y = y-altura_imagem+(altura_imagem/2)

    pdf.drawImage(image=path, x=x, y=y, width=largura_imagem, height=altura_imagem, preserveAspectRatio=True, mask='auto', showBoundary=showBoundary)

    # print(f'image "{nome_imagem}" proporcionada: {proporcao}')

    return proporcao

def cabecalho(pdf: canvas.Canvas, altura_pag, largura_pag):
    margens = 15 # Definindo as margens
    # Margens do relatorio
    frame0=Frame(margens,margens,largura_pag-margens*2,altura_pag-margens*2, showBoundary=1)
    frame0.addFromList([], pdf)
    
    # Imagem Pref, 1 parte do cabecalho
    cabecalho1 = inserir_imagem('cabeçalho esq.png', margens+5, altura_pag - (margens+5), 180)

    # Calculo para alinhar à Imagem Pref, 1* parte do cabecalho
    metade_cabecalho1 = altura_pag - (margens+5) - cabecalho1[1] + cabecalho1[1]/2
   
    # Imagem SEMINF, 2 parte do cabecalho
    inserir_imagem('logo-secretaria-seminf-header.png', largura, metade_cabecalho1, 120,centered='CENTER')

    # Criar um estilo personalizado com a fonte desejada
    estilo_personalizado = ParagraphStyle(
        "estilo_personalizado",
        parent=getSampleStyleSheet()["Heading4"],  # Copie as configurações de estilo padrão
        fontName="Courier-Bold",  # Substitua pelo nome da fonte desejada
        fontSize=12
    )

    # Endereço, 3 parte do cabecalho
    p = Paragraph('Rua: Gabriel Gonçalves nº 351 – Aleixo - CEP 69060–010\nTel.: 3642-3064 - Manaus-AM\nwww.manaus.am.gov.br', estilo_personalizado)
    p.wrapOn(pdf, 200, 48)
    p.drawOn(pdf, x=largura_pag-(margens+5)-p.width, y=metade_cabecalho1-p.height+(p.height/2))

    altura_pag = altura_pag - (margens + 5) - cabecalho1[1]

    return centralizar_texto(pdf, altura_pag-14-5, "INVENTÁRIO DE BENS IMÓVEIS", 'Helvetica-Bold', 14, 15)
    # print(f"Cabeçalho do Relatório de inventário gerado em teste.pdf")

pdf=canvas.Canvas("teste.pdf", pagesize=landscape(A4))
largura, altura = landscape(A4)
# print(f'largura X altura: {largura} X {altura}')
flow_obj = []
# pdf.translate(cm, cm)

# Covertendo a planilha em uma lista com os dados
# data_row = pd.read_excel(r"planilha_com_valores_aleatorios.xlsx").values.tolist()
data = pd.read_excel(r"C:\Users\luan.pinto\Desktop\Códigos\Projeto - Controle de obras\ARQUIVOS\obras_2021.xlsx")
data_row = data.values.tolist()

# Paginação
count=len(data_row)
print(count)
# # print(f'count: {count}')
# num=int(count/40)
# # print(f'num: {num}')
# a=int(count/num)
# # print(f'a: {a}')
num=20 # Numero de linhas em uma página
count_new=[0]
if count <= num: count_new, num=[0, count], count
x=0
while (x+int(count/(int(count/num)))) < count:
    x=x+int(count/(int(count/num)))
    count_new.append(x)

altura = cabecalho(pdf=pdf, altura_pag=altura, largura_pag=largura)

frame1=Frame(15,15,360,280, showBoundary=1)
styles=getSampleStyleSheet()
text='This is database report for pagination <b></b>,<b></b>'
t1=Paragraph(text,style=styles['Normal'])
flow_obj.append(t1)
flow_obj.append(Spacer(6,6))

text='data generated at '+str(datetime.datetime.now())
t2=Paragraph(text,style=styles['Normal'])
flow_obj.append(t2)
flow_obj.append(Spacer(6,6))
frame1.addFromList(flow_obj, pdf)

tamanho_fonte = 10
pdf.showPage()
print('count_new:',count_new)
for j in range(len(count_new)-1):
    print('j:',j)
    largura, altura = landscape(A4)
    altura = cabecalho(pdf, altura, largura)
    frame=Frame(x1=15,y1=15,width=largura-30,height=altura,showBoundary=0)
    data1=[data.columns]
    
    for row in data_row[count_new[j]:count_new[j+1]]: data1.append(row)
    
    # table=Table(data1,colWidths=[20, 50, 50, 100, 50, 100, 50, 80, 50], rowHeights=[15 for i in range(len(data1))])
    table=Table(data1, rowHeights=[15 for i in range(len(data1))])
    # ts=TableStyle([("GRID",(0,0),(-1,-1),2,colors.black),
    #                ("BACKGROUND",(0,0),(-1,0),colors.yellow),
    #                ("BACKGROUND",(0,1),(-1,-1),colors.lightblue),
    #                ("SIZE",(0,0),(-1,-1),6,colors.yellow),
    #                ("ALIGN",(0,0),(-1,-1),"CENTER")])
    style = TableStyle([
    ('BACKGROUND', (0, 0), (-1, 0), colors.grey),  # Cor de fundo do cabeçalho
    ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),  # Cor do texto do cabeçalho
    ('ALIGN', (0, 0), (-1, -1), 'CENTER'),  # Alinhamento no centro
    ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),  # Fonte em negrito para o cabeçalho
    ('BOTTOMPADDING', (0, 0), (-1, 0), 12),  # Espaçamento inferior do cabeçalho
    ('BACKGROUND', (0, 1), (-1, -1), colors.beige),  # Cor de fundo das células de dados
    ('GRID', (0, 0), (-1, -1), 1, colors.black),  # Grade para todas as células
    ('FONTNAME', (0, 1), (-1, -1), 'Helvetica'),  # Fonte para as células de dados
    ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),  # Alinhamento vertical no meio das células
    ('FONTSIZE', (0, 0), (-1, -1), 4),  # Tamanho da fonte para todas as células
    ])

    table.setStyle(style)
    flow_obj.append(Spacer(8,8))
    flow_obj.append(table)
    frame.addFromList(drawlist=flow_obj, canv=pdf)
    pdf.showPage()


pdf.save()

19
count_new: [0, 19]
j: 0


In [768]:
data

,OBJETO,CONTRATO N°,EMPRESA,SITUAÇÃO DA OBRA,VALOR DO CONTRATO (R$),VALOR EMPENHADO (R$),VALOR ANULADO (R$),VALOR TOTAL (R$),EMPENHO,UNIDADE ORÇAMENTARIA,PROGRAMA DE TRABALHO,FONTE DE RECURSO,NATUREZA DESPESA,CONTA CONTÁBIL
0,CONSTRUÇÃO DE TALUDE - RUA BELA VISTA C/ BECO ...,018/2021,RDS ENGENHARIA DE OBRAS E EMPREENDIMENTOS IMOB...,PARADA TÉCNICA,271269.96,271269.96,NaN,NaN,2021NE00542,27101 - SECRETARIA MUNICIPAL DE INFRAESTRUTURA,15451001410950000 - CONSTRUÇÃO DE ESTRUTURAS D...,01000000 - RECURSOS ORDINÁRIOS,44905117 - OBRAS DE INFRAESTRUTURA,123210517 - OBRAS DE INFRAESTRUTURA DE REVITAL...
1,CONSTRUÇÃO DE TALUDE - RUA BELA VISTA C/ BECO ...,018/2021,RDS ENGENHARIA DE OBRAS E EMPREENDIMENTOS IMOB...,PARADA TÉCNICA,271269.96,NaN,32509.05,ANULAÇÃO PARCIAL,2021NE01567,27101 - SECRETARIA MUNICIPAL DE INFRAESTRUTURA,15451001410950000 - CONSTRUÇÃO DE ESTRUTURAS D...,01000000 - RECURSOS ORDINÁRIOS,44905117 - OBRAS DE INFRAESTRUTURA,123210517 - OBRAS DE INFRAESTRUTURA DE REVITAL...
2,"EXECUÇÃO DE OBRA DE REVITALIZAÇÃO DE EROSÃO, L...",020/2021,CONSTRUTORA SOMA LTDA.,CONCLUÍDA,4465537.18,4465537.18,NaN,NaN,2021NE00734,56701 - FUNDO MUNICIPAL DE DESENVOLVIMENTO URBANO,15451014210940000 - OBRAS DE INFRAESTRUTURA UR...,02100356 - RECURSOS DO PROMINF / MANAUS / FINI...,44905117 - OBRAS DE INFRAESTRUTURA,123210517 - OBRAS DE INFRAESTRUTURA DE REVITAL...
3,"EXECUÇÃO DE OBRA DE REVITALIZAÇÃO DE EROSÃO, L...",020/2021,CONSTRUTORA SOMA LTDA.,CONCLUÍDA,4465537.18,NaN,39791.08,NaN,2021NE01561,56701 - FUNDO MUNICIPAL DE DESENVOLVIMENTO URBANO,15451014210940000 - OBRAS DE INFRAESTRUTURA UR...,02100356 - RECURSOS DO PROMINF / MANAUS / FINI...,44905117 - OBRAS DE INFRAESTRUTURA,123210517 - OBRAS DE INFRAESTRUTURA DE REVITAL...
4,OBRAS E SERVIÇOS DE ENGENHARIA PARA A CONSTRUÇ...,022/2021,TECNOARTE DA AMAZÔNIA CONSTRUÇÃO CIVIL LTDA - EPP,CONCLUÍDA,374643.14,347643.14,NaN,NaN,2021NE00756,56701 - FUNDO MUNICIPAL DE DESENVOLVIMENTO URBANO,15451014210940000 - OBRAS DE INFRAESTRUTURA UR...,02100356 - RECURSOS DO PROMINF / MANAUS / FINI...,44905117 - OBRAS DE INFRAESTRUTURA,123210517 - OBRAS DE INFRAESTRUTURA DE REVITAL...
5,OBRAS E SERVIÇOS DE ENGENHARIA PARA A CONSTRUÇ...,022/2021,TECNOARTE DA AMAZÔNIA CONSTRUÇÃO CIVIL LTDA - EPP,CONCLUÍDA,374643.14,NaN,77471.25,NaN,2021NE01570,56701 - FUNDO MUNICIPAL DE DESENVOLVIMENTO URBANO,15451014210940000 - OBRAS DE INFRAESTRUTURA UR...,02100356 - RECURSOS DO PROMINF / MANAUS / FINI...,44905117 - OBRAS DE INFRAESTRUTURA,123210517 - OBRAS DE INFRAESTRUTURA DE REVITAL...
6,OBRAS E SERVIÇOS DE ENGENHARIA PARA A CONSTRUÇ...,023/2021,AMAZONCRETO CONSTRUÇÕES EIRELI,CONCLUÍDA,1733790.85,1733790.85,NaN,NaN,2021NE00809,56701 - FUNDO MUNICIPAL DE DESENVOLVIMENTO URBANO,15451014210940000 - OBRAS DE INFRAESTRUTURA UR...,06100358 - REQUALIFICA III/MANAUS - EXERCÍCIOS...,44905117 - OBRAS DE INFRAESTRUTURA,123210517 - OBRAS DE INFRAESTRUTURA DE REVITAL...
7,OBRAS E SERVIÇOS DE ENGENHARIA PARA A CONSTRUÇ...,023/2021,AMAZONCRETO CONSTRUÇÕES EIRELI,CONCLUÍDA,1733790.85,NaN,468387.91,NaN,2021NE01566,56701 - FUNDO MUNICIPAL DE DESENVOLVIMENTO URBANO,15451014210940000 - OBRAS DE INFRAESTRUTURA UR...,06100358 - REQUALIFICA III/MANAUS - EXERCÍCIOS...,44905117 - OBRAS DE INFRAESTRUTURA,123210517 - OBRAS DE INFRAESTRUTURA DE REVITAL...
8,SERVIÇOS TÉCNICOS ESPECIALIZADOS EM MANUTENÇÃO...,025/2021,EDS CONSTRUÇÕES E SERVIÇOS LTDA,CONCLUÍDA,3488673.60,290722.80,NaN,NaN,2021NE01192,27101 - SECRETARIA MUNICIPAL DE INFRAESTRUTURA,"15451001620200000 - MANUTENÇÃO, REFORMA E REVI...",01000000 - RECURSOS ORDINÁRIOS,44905117 - OBRAS DE INFRAESTRUTURA,123210517 - OBRAS DE INFRAESTRUTURA DE REVITAL...
9,SERVIÇOS TÉCNICOS ESPECIALIZADOS EM MANUTENÇÃO...,025/2021,EDS CONSTRUÇÕES E SERVIÇOS LTDA,CONCLUÍDA,3488673.60,NaN,290722.80,NaN,2021NE01565,27101 - SECRETARIA MUNICIPAL DE INFRAESTRUTURA,"15451001620200000 - MANUTENÇÃO, REFORMA E REVI...",01000000 - RECURSOS ORDINÁRIOS,44905117 - OBRAS DE INFRAESTRUTURA,123210517 - OBRAS DE INFRAESTRUTURA DE REVITAL

In [777]:
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Table, TableStyle
from reportlab.lib.styles import getSampleStyleSheet
import pandas as pd

# DataFrame de exemplo (substitua pelo seu próprio DataFrame)
data = {
    'Nome': ['João', 'Maria', 'Carlos'],
    'Idade': [30, 25, 35],
}

df = pd.read_excel(r"C:\Users\luan.pinto\Desktop\Códigos\Projeto - Controle de obras\ARQUIVOS\obras_2021.xlsx")

# Função para criar uma tabela a partir do DataFrame
def create_table_from_dataframe(dataframe):
    table_data = [dataframe.columns.tolist()]  # Cabeçalho da tabela
    table_data.extend(dataframe.values.tolist())  # Dados do DataFrame
    return table_data

# Crie um PDF
pdf_file = "relatorio.pdf"
document = SimpleDocTemplate(pdf_file, pagesize=letter)

# Estilos para parágrafos
styles = getSampleStyleSheet()
style_normal = styles['Normal']

# Lista de elementos do PDF
elements = []

contratos_processados = set()  # Crie um conjunto para rastrear os contratos processados

# Itera sobre as linhas do DataFrame
# for index, row in df.iterrows():
for n_contrato in df["CONTRATO N°"]:
    # Cria um parágrafo com informações
    info_paragraph = Paragraph(f"Objeto: {row['OBJETO']}\nN° do Contrato{row['CONTRATO N°']}\t Situação da Obra: {row['SITUAÇÃO DA OBRA']}\t Vlr. do Contrato: {row['VALOR DO CONTRATO (R$)']}", style_normal)
    elements.append(info_paragraph)

    # Cria uma tabela com base nas informações do DataFrame
    if n_contrato not in contratos_processados:
        print(f'++++++++++++ {n_contrato} ++++++++++++')
        corte_df = df[df['CONTRATO N°'] == n_contrato]
        # display(corte_df)
        t_corte_df = corte_df[['VALOR EMPENHADO (R$)', 'VALOR ANULADO (R$)',
       'VALOR TOTAL (R$)', 'EMPENHO', 'UNIDADE ORÇAMENTARIA',
       'PROGRAMA DE TRABALHO', 'FONTE DE RECURSO', 'NATUREZA DESPESA',
       'CONTA CONTÁBIL']]
        # display(t_corte_df)
        contratos_processados.add(n_contrato)

    table_data = create_table_from_dataframe(t_corte_df)
    table = Table(table_data)#, colWidths=[100, 100])  # Ajuste o tamanho das colunas conforme necessário
    table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), (0.5, 0.5, 0.5)),  # Cor de fundo do cabeçalho
        ('TEXTCOLOR', (0, 0), (-1, 0), (1, 1, 1)),  # Cor do texto do cabeçalho
        ('BACKGROUND', (0, 1), (-1, -1), (0.9, 0.9, 0.9)),  # Cor de fundo das células de dados
        ('GRID', (0, 0), (-1, -1), 1, (0, 0, 0)),  # Grade para todas as células
    ]))
    elements.append(table)

# Adicione os elementos ao documento PDF
document.build(elements)

print(f"O arquivo '{pdf_file}' foi criado com sucesso.")


++++++++++++ 018/2021 ++++++++++++
++++++++++++ 020/2021 ++++++++++++
++++++++++++ 022/2021 ++++++++++++
++++++++++++ 023/2021 ++++++++++++
++++++++++++ 025/2021 ++++++++++++
++++++++++++ 036/2021 ++++++++++++
++++++++++++ 037/2021 ++++++++++++
++++++++++++ 038/2021 ++++++++++++
++++++++++++ 039/2021 ++++++++++++
++++++++++++ 040/2021 ++++++++++++
++++++++++++ Emenda Parlamenta n° 137/2021 ++++++++++++
O arquivo 'relatorio.pdf' foi criado com sucesso.


In [786]:
table_data_copy = table_data[1]
table_data_copy#.split(' - ')

for i in table_data_copy:
    if i is str and ' - ' in a: i = i.split(' - ')[0]
    print(a)


40
40
40
40
40
40
40
40
40


#### Codigo Principal

In [697]:
from reportlab.lib.pagesizes import A4
from reportlab.pdfgen import canvas
from reportlab.platypus import Paragraph
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle

from datetime import datetime

class Create_PDF():
    def __init__(self, nome_arquivo="relatorio_inventario.pdf") -> None:
        # self.nome_arquivo = nome_arquivo
        self._c = canvas.Canvas(nome_arquivo, pagesize=A4)
        # Obter a largura da página
        self._largura_pagina, self._altura_pagina = A4
        
    def get_nome_arquivo(self): return self._c._filename

    def centralizar_texto(self, y_titulo, titulo, fontName, fontSize, espacamento=30):         
        # Calcular a posição X para centralizar o título
        x_titulo = (self._largura_pagina) / 2

        # Configurar o título do relatório
        self._c.setFont(fontName, fontSize)

        # Desenhar o título centralizado
        self._c.drawCentredString(x_titulo, y_titulo, titulo)

        self._altura_pagina = y_titulo - espacamento  # Iniciar logo abaixo do título

    def gerar_relatorio_inventario(self, dados_inventario):
        # Configurar os dados do inventário
        self._c.setFont("Helvetica", 12)
        y = self._altura_pagina  # Posição vertical inicial para os dados
        for item in dados_inventario:
            self._c.drawString(100, y, f"Item: {item['nome']}")
            self._c.drawString(300, y, f"Quantidade: {item['quantidade']}")
            y -= 20  # Mover para a próxima linha

    def salvar_pdf(self): self._c.save()

    def inserir_imagem(self, imagem_path, ):
        # Inserir a imagem
        imagem_x = 400  # Posição X da imagem
        imagem_y = self._altura_pagina - 150  # Posição Y da imagem
        # largura_imagem = 100  # Largura da imagem
        # altura_imagem = 100  # Altura da imagem

        self._c.drawImage(imagem_path, x=imagem_x, y=imagem_y, width=200, mask='auto', preserveAspectRatio=True)
        # self._c.drawImage(imagem_path, x=imagem_x, y=imagem_y, mask='auto')

    def cabecalho(self):
        self._c.drawImage(image=r'C:\Users\luan.pinto\Desktop\Códigos\Projeto - Controle de obras\IMAGE\cabeçalho esq.png', x=15, y=self._altura_pagina - 195, width=180, mask='auto', preserveAspectRatio=True)
        self._c.drawImage(image=r'C:\Users\luan.pinto\Desktop\Códigos\Projeto - Controle de obras\IMAGE\cabeçalho meio.png', x=225, y=self._altura_pagina - 95, width=140, mask='auto', preserveAspectRatio=True)

        # Criar um estilo personalizado com a fonte desejada
        estilo_personalizado = ParagraphStyle(
            "estilo_personalizado",
            parent=getSampleStyleSheet()["Heading4"],  # Copie as configurações de estilo padrão
            fontName="Courier-Bold",  # Substitua pelo nome da fonte desejada
            fontSize=12
        )

        p = Paragraph('Rua: Gabriel Gonçalves nº 351 – Aleixo - CEP 69060–010\nTel.: 3642-3064 - Manaus-AM\nwww.manaus.am.gov.br', estilo_personalizado)
        p.wrapOn(self._c, 200, 100)
        p.drawOn(self._c, 380, self._altura_pagina - 75)

        # self.centralizar_texto("INVENTÁRIO DE BENS IMÓVEIS")
        # self.gerar_relatorio_inventario(inventario)
        print(f"Cabeçalho do Relatório de inventário gerado em {self.get_nome_arquivo()}")

    def rodape(self):
        # Defina o estilo do texto do rodapé
        estilo_rodape = ParagraphStyle(
            'rodape',
            parent=getSampleStyleSheet()['Normal'],
            fontSize=10,
            alignment=1
        )

        # Obtenha a data atual
        data_atual = datetime.now().strftime('%d/%m/%Y - %H:%M:%S')

        # Obtenha o número da página atual
        numero_pagina = self._c.getPageNumber()

        # Defina o texto do rodapé
        texto_rodape = f'{data_atual}  |  Página {numero_pagina}'

        # Crie um objeto Paragraph com o texto do rodapé e o estilo definido
        paragrafo_rodape = Paragraph(texto_rodape, estilo_rodape)

        # Desenhe o parágrafo do rodapé no self._c
        paragrafo_rodape.wrap(self._largura_pagina, 0)
        paragrafo_rodape.drawOn(self._c, 0, 20)

# if __name__ == "__main__":
inventario = [
    {"nome": "Produto A", "quantidade": 10},
    {"nome": "Produto B", "quantidade": 20},
    {"nome": "Produto C", "quantidade": 15},
    {"nome": "Produto D", "quantidade": 25},
    {"nome": "Produto E", "quantidade": 20},
    {"nome": "Produto F", "quantidade": 30},
    {"nome": "Produto G", "quantidade": 25},
    {"nome": "Produto H", "quantidade": 35},
]

pdf = Create_PDF()
for i in range(3):
    pdf.cabecalho()
    pdf.rodape()
    pdf._c.drawRightString(pdf._largura_pagina/2, 500, 'Nome de usuario: Luan Lucas')
    # Adiciona uma nova folha
    pdf._c.showPage()
pdf.salvar_pdf()


Cabeçalho do Relatório de inventário gerado em relatorio_inventario.pdf
Cabeçalho do Relatório de inventário gerado em relatorio_inventario.pdf
Cabeçalho do Relatório de inventário gerado em relatorio_inventario.pdf


#### Fontes disponíveis no ReportLab

In [ ]:
for font in pdf._c.getAvailableFonts():
    print(font)

Courier
Courier-Bold
Courier-BoldOblique
Courier-Oblique
Helvetica
Helvetica-Bold
Helvetica-BoldOblique
Helvetica-Oblique
Symbol
Times-Bold
Times-BoldItalic
Times-Italic
Times-Roman
ZapfDingbats


#### Testando a Spire

#### Usando a lib xlsx2pdf
### Requisitos básicos
- ### Java Version: 1.8 or higher

In [1]:
import jpype
import asposecells
jpype.startJVM()
from asposecells.api import Workbook

# Open an existing file
workbook = Workbook(r'C:\Users\luan.pinto\Desktop\Códigos\Projeto - Controle de obras\ARQUIVOS\obras_2021.xlsx')

# Set cell value
workbook.getWorksheets().get(0).getCells().get("A1").putValue("Aspose Cells")

# Save as XLS
workbook.save("output.xls")

# Save as XLSX
workbook.save("output.xlsx")

# Save as ODS
workbook.save("output.ods")

jpype.shutdownJVM()

ImportError: Failed to import 'com.aspose'